In [1]:
import epidag as dag
import complexism as cx
import complexism.fn as cmd

ModuleNotFoundError: No module named 'epidag'

In [2]:
from itertools import groupby
from enum import Enum, auto
from abc import ABCMeta, abstractmethod

In [3]:
class Event:
    def __init__(self, todo, time):
        self.Todo = todo
        self.Time = time
        self.Message = todo

In [6]:
req = Request(Event('sleep', 5), 'Me', 'Sheffield')
print(req)
req= req.up_scale('Yorkshire')
print(req)
exp = req.to_exposure()
print(exp)
exp = exp.up_scale('England')
print(exp)
adr, exp = exp.down_scale()
print(adr, exp)

try:
    exp.down_scale()
except AttributeError as e:
    print('Error: ', e)

Request(Me want to do sleep in Sheffield when t=5.000)
Request(Me want to do sleep in Sheffield@Yorkshire when t=5.000)
Exposure(Me did sleep in Sheffield@Yorkshire)
Exposure(Me did sleep in Sheffield@Yorkshire@England)
England Exposure(Me did sleep in Sheffield@Yorkshire)
Error:  Affected scale reached


In [27]:
class Status(Enum):
    TO_COLLECT = auto()  # collect
    TO_VALIDATE = auto()
    TO_EXECUTE = auto()


class Schedule:
    def __init__(self, loc):
        self.Location = loc
        self.RequestList = list()
        self.ExposureList = list()
        self.Time = float('inf')
        self.Status = Status.TO_COLLECT

    def __gt__(self, ot):
        return self.Time > ot.Time

    def __le__(self, ot):
        return self.Time <= ot.Time

    def __eq__(self, ot):
        return self.Time == ot.Time

    def __len__(self):
        return len(self.RequestList) + len(self.ExposureList)

    def __iter__(self):
        return iter(self.ExposureList + self.RequestList)

    # Collection stage
    
    def append_request(self, req: Request):
        """
        Append a request
        :param req: request to be execute
        :return: true if the request is the nearest
        """
        ti = req.When
        if ti < self.Time:
            self.RequestList = [req]
            self.Time = ti
            return True
        elif ti == self.Time:
            self.RequestList.append(req)
            return True
        return False

    def append_request_from_source(self, event, who):
        req = Request(event, who, self.Location)
        return self.append_request(req)

    def append_exposure(self, exp):
        """
        Append an exposure
        :param exp: message to be exposed
        """
        self.ExposureList.append(exp)

    def up_scale_requests(self, loc):
        return [req.up_scale(loc) for req in self.RequestList]
    
    def up_scale_exposures(self, loc):
        return [exp.up_scale(loc) for exp in self.ExposureList]

    def append_lower_schedule(self, lower):
        ti = lower.Time
        if ti < self.Time:
            self.RequestList = lower.up_scale_requests(self.Location)
            if lower.ExposureList:
                self.ExposureList = lower.up_scale_exposures(self.Location)
            self.Time = ti
        elif ti == self.Time:
            self.RequestList += lower.up_scale_requests(self.Location)
            if lower.ExposureList:
                self.ExposureList += lower.up_scale_exposures(self.Location)
        else:
            return
        lower.collection_completed()
    
    # After validation
        
    def approve(self, req_set, exp_set):
        self.RequestList = req_set
        self.ExposureList = exp_set
        self.ExposureList += [req.to_exposure() for req in req_set]

    def pop_approved_lower_requests(self):
        gp = groupby(self.Requests, lambda x: x.reached())
        lower, current = list(), list()
        for k, g in gp:
            if k:
                current += list(g)
            else:
                lower += [req.down_scale() for req in g]

        if lower:
            pop = {k: [req for _, req in g] for k, g in groupby(lower, lambda x: x[1].Group)}
        else:
            pop = dict()

        self.Requests = current

        if (not self.Requests) and (not lower):
            self.execution_completed()

        return pop

    def emit_exposures(self):
        if self.Status is Status.TO_EXECUTE:
            return self.ExposureList
    
    def disapprove(self, req_set, exp_set):
        pass

    def pop_disapproved_lower_requests(self):
        pass

    def is_waiting(self):
        return self.Status is Status.TO_COLLECT
    
    def collection_completed(self):
        self.Status = Status.TO_VALIDATE

    def validation_completed(self):
        if self.is_empty():
            self.execution_completed()
        else:
            self.Status = Status.TO_EXECUTE

    def execution_completed(self):
        self.Status = Status.TO_COLLECT
        self.ExposureList.clear()
        self.RequestList.clear()
        self.Time = float('inf')
        
    def updated(self):
        self.execution_completed()

    def is_empty(self):
        return not self.RequestList and not self.ExposureList

    def __repr__(self):
        return 'Status: {}, Req: {}, Exp: {}, Next Event Time: {}'.format(
            self.Status,
            len(self.RequestList), len(self.ExposureList), self.Time)

In [8]:
class AbsModel(metaclass=ABCMeta):
    def __init__(self, name):
        self.Name = name
        self.Scheduler = Schedule(name)
        
    @property
    def Next(self):
        if self.Requests.is_empty():
            self.find_next()
        return self.Scheduler

    @property
    def TTE(self):
        return self.Scheduler.Time

    def drop_next(self):
        self.Scheduler.execution_completed()

    @abstractmethod
    def find_next(self):
        pass

    @abstractmethod
    def fetch_requests(self, rs):
        pass

    @abstractmethod
    def execute_requests(self):
        pass

    @abstractmethod
    def do_request(self, req):
        pass
    
class LeafModel(AbsModel, metaclass=ABCMeta):
    def __init__(self, name, pc=None, obs: Observer=None):
        AbsModel.__init__(self, name, pc, obs)

    def fetch_requests(self, rs):
        self.Requests.clear()
        self.Requests.append_requests(rs)

    def execute_requests(self):
        for req in self.Requests:
            self.do_request(req)
        self.drop_next()


class BranchModel(AbsModel, metaclass=ABCMeta):
    def __init__(self, name):
        AbsModel.__init__(self, name)

    def preset(self, ti):
        for v in self.all_models().values():
            v.preset(ti)
        self.reset_impulse(ti)

    def reset(self, ti):
        for v in self.all_models().values():
            v.reset(ti)
        self.reset_impulse(ti)

    @abstractmethod
    def reset_impulse(self, ti):
        pass

    def select(self, mod):
        return self.get_model(mod)

    def select_all(self, sel):
        return ModelSelector(self.all_models()).select_all(sel)

    def fetch_requests(self, res):
        self.Requests.clear()
        self.Requests.append_requests(res)
        self.pass_down()

    def pass_down(self):
        res = self.Requests.pop_lower_requests()

        for k, v in res.items():
            self.select(k).fetch_requests(v)

    def execute_requests(self):
        for req in self.Requests:
            self.do_request(req)

        ti = self.TTE
        for v in self.all_models().values():
            if v.TTE == ti:
                v.execute_requests()
        self.drop_next()

    @abstractmethod
    def do_request(self, req):
        pass

In [2]:
psc = """
PCore pSIR {
    beta = 0.4
    gamma = 0.5
    Infect ~ exp(beta)
    Recov ~ exp(0.5)
    Die ~ exp(0.02)
}
"""

dsc = """
CTBN SIR {
    life[Alive | Dead]
    sir[S | I | R]

    Alive{life:Alive}
    Dead{life:Dead}
    Inf{life:Alive, sir:I}
    Rec{life:Alive, sir:R}
    Sus{life:Alive, sir:S}

    Die -> Dead # from transition Die to state Dead by distribution Die
    Sus -- Infect -> Inf
    Inf -- Recov -> Rec

    Alive -- Die # from state Alive to transition Die
}
"""

bn = cx.read_pc(psc)
dc = cx.read_dc(dsc)
sm = dag.as_simulation_core(bn, hie={'city': ['agent'], 'agent': ['Recov', 'Die', 'Infect']})

Three types of agents are supported by PyComplexism

- StSpAgent (State space agent): agents with states from state-space models
- DiffEqAgent (Differential equation agent): agents with inner dynamics from differential equations
- GenericAgent: agents allow fully customisation

Building an agent-based model
1. Design a parameter model
2. Design a dynamic model: ctmc, ctbn, or none
3. Design agents: state-space, diffeq, generic
4. Design population structure
5. Add mods

In [3]:
model_blueprint = {
    'agent': {
        'prefix': 'agent',
        'group': 'agent',
        'exo': {},
        'type': 'stsp',
        'dynamics': 'SIR',
        'default': {
            'st': 'Sus'
        }
    },
    'population': {
        'networks': {}
    },
    'mods': {
        
    }
}

In [4]:
model_blueprint = {
    'agent': {
        'prefix': 'Helen',
        'group': 'agent',
        'exo': {},
        'type': 'stsp',
        'dynamics': 'SIR',
        'default': {
            'st': 'Sus'
        }
    }
}

Name = 'M1'
Gene = sm.generate()
proto = Gene.get_prototype('agent')
stsp = dc.generate_model('agent', **proto.get_samplers())
ag = cx.StSpAgent('Helen', stsp['Sus'], proto.get_sibling('Helen'))
model = cx.SingleIndividualABM(Name, ag)
cx.simulate(model, None, 0, 10, 1)

,Name,State,Infect,Recov
Time,,,,
0.0,Helen,Sus,0.0,0.0
1.0,Helen,Inf,1.0,0.0
2.0,Helen,Inf,0.0,0.0
3.0,Helen,Inf,0.0,0.0
4.0,Helen,Rec,0.0,1.0
5.0,Helen,Rec,0.0,0.0
6.0,Helen,Rec,0.0,0.0
7.0,Helen,Rec,0.0,0.0
8.0,Helen,Rec,0.0,0.0


In [5]:
model_blueprint = {
    'agent': {
        'prefix': 'agent',
        'group': 'agent',
        'exo': {},
        'type': 'stsp',
        'dynamics': 'SIR',
        'default': {
            'st': 'Sus'
        }
    },
    'population': {
        'networks': {}
    },
    'mods': {
        
    }
}


y0 = [
    {'n': 95, 'attributes': {'st': 'Sus'}},
    {'n': 5, 'attributes': {'st': 'Inf'}},
]

Name = 'M1'
Gene = sm.generate()
eve = cx.StSpBreeder('Ag ', 'agent', Gene, dc)
pop = cx.Population(eve)
# model = cx.SingleIndividualABM(Name, ag)
# cx.simulate(model, None, 0, 10, 1)
model = cx.AgentBasedModel(Name, Gene, pop)
model.add_observing_event(pop.Eve.DCore.Transitions['Infect'])
model.add_observing_event(pop.Eve.DCore.Transitions['Recov'])
model.add_observing_event(pop.Eve.DCore.Transitions['Die'])

def f(model, tab, ti):
    tab['Sus'] = model.Population.count(st='Sus')
    tab['Inf'] = model.Population.count(st='Inf')
    tab['Rec'] = model.Population.count(st='Rec')
    tab['Alive'] = model.Population.count(st='Alive')
    tab['Dead'] = model.Population.count(st='Dead')
    

model.add_observing_function(f)

cx.simulate(model, y0, 0, 10, 1)

,Sus,Inf,Rec,Alive,Dead,"Tr(Name: Infect, To: Inf, By: exp(beta))","Tr(Name: Recov, To: Rec, By: exp(0.5))","Tr(Name: Die, To: Dead, By: exp(0.02))"
Time,,,,,,,,
0.0,95,5,0,100,0,0.0,0.0,0.0
1.0,63,26,11,100,0,32.0,11.0,0.0
2.0,36,34,26,96,4,25.0,15.0,4.0
3.0,22,30,42,94,6,14.0,17.0,2.0
4.0,14,24,53,91,9,8.0,13.0,3.0
5.0,10,14,66,90,10,4.0,14.0,1.0
6.0,6,13,70,89,11,4.0,5.0,1.0
7.0,3,10,75,88,12,2.0,5.0,1.0
8.0,3,6,79,88,12,0.0,4.0,0.0


In [8]:
y0 = [
    {'n': 995, 'attributes': {'st': 'Sus'}},
    {'n': 5, 'attributes': {'st': 'Inf'}},
]

Name = 'M1'
Gene = sm.generate()
eve = cx.StSpBreeder('Ag ', 'agent', Gene, dc)
pop = cx.Population(eve)
# model = cx.SingleIndividualABM(Name, ag)
# cx.simulate(model, None, 0, 10, 1)
model = cx.StSpAgentBasedModel(Name, Gene, pop)

for tr in ['Infect', 'Recov', 'Die']:
    model.add_observing_transition(tr)

for st in ['Sus', 'Inf', 'Rec', 'Alive', 'Dead']:
    model.add_observing_state(st)

cx.simulate(model, y0, 0, 10, 1)

,Sus,Inf,Rec,Alive,Dead,Infect,Die,Recov
Time,,,,,,,,
0.0,995,5,0,1000,0,0.0,0.0,0.0
1.0,654,249,78,981,19,324.0,19.0,78.0
2.0,434,311,214,959,41,210.0,22.0,139.0
3.0,276,286,376,938,62,149.0,21.0,167.0
4.0,185,220,514,919,81,86.0,19.0,145.0
5.0,115,165,616,896,104,66.0,23.0,115.0
6.0,72,134,677,883,117,43.0,13.0,71.0
7.0,51,101,704,856,144,19.0,27.0,46.0
8.0,30,66,733,829,171,18.0,27.0,50.0


In [ ]:

sm1 = dag.as_simulation_core(bn)
sm1.deep_print()

In [ ]:
mc_bp = BlueprintABM('AB_abm', 'pAB', 'AB_mc')
mc_bp.set_observations(states=['N', 'A', 'B', 'AB'])
pc = pc_bp.get_simulation_model().sample_core()
dc = dc_bp.generate_model(pc=pc)

model = mc_bp.generate('A', pc=pc, dc=dc)
out = simulate(model, y0={'N': 500}, fr=0, to=10)
model.output(mid=False)

In [ ]:

par_script = """ 
PCore pSIR{
    transmission_rate = 1.5/200
    rec_rate ~ triangle(0.1, 0.2, 0.3)
    beta ~ exp(transmission_rate)
    gamma ~ exp(rec_rate)
    Die ~ exp(0.2)
}
"""

dc_ctbn_script = '''
CTBN SIR_BN {
    life[Alive | Dead]
    sir[S | I | R]
    
    Alive{life:Alive}
    Dead{life:Dead}
    Inf{life:Alive, sir:I}
    Rec{life:Alive, sir:R}
    Sus{life:Alive, sir:S}

    Die -> Dead # from transition Die to state Dead by distribution Die
    Sus -- Infect_Net(beta) -> Inf
    Sus -- Infect(beta) -> Inf 
    Inf -- Recov(gamma) -> Rec
    
    Alive -- Die # from state Alive to transition Die
}
'''

da = Director()
da.read_pc(par_script)
da.read_dc(dc_ctbn_script)

In [ ]:
pc = da.get_pc('pSIR')
pc = pc.sample_core()
pc.Locus

In [ ]:
cfd = da.new_mc('ABM_M2', 'ABM', tar_pc='pSIR', tar_dc='SIR_BN')
cfd.add_behaviour('cycle', be_type='Reincarnation', s_birth = 'Sus', s_death = 'Dead')
cfd.add_behaviour('foi', be_type='ComFDShock', s_src='Inf', t_tar = 'Infect', dt=0.5)
cfd.add_behaviour('step', be_type='TimeStep', ts=[0.5, 4], ys=[1, 0.1, 5], t_tar = 'Infect')

cfd.set_observations(states=['Sus', 'Inf', 'Rec'])

mod, out = da.simulate('ABM_M2', y0={'Sus': 95, 'Inf': 5}, fr=0 ,to=10, dt=.01)
out.plot()
plt.show()

In [ ]:
cfd = da.new_mc('ABM_M2', 'ABM', tar_pc='pSIR', tar_dc='SIR_BN')
cfd.add_network('IDU', 'BA', m=2)
cfd.add_behaviour('cycle', be_type='Reincarnation', s_birth = 'Sus', s_death = 'Dead')
cfd.add_behaviour('transmission', be_type='NetShock', s_src = 'Inf', t_tar = 'Infect_Net', net='IDU')
cfd.add_behaviour('foi', be_type='ForeignShock', t_tar = 'Infect')

cfd.set_observations(states=['Inf'], behaviours=['foi'])

cfd = da.new_mc('ABM_M5', 'ABM', tar_pc='pSIR', tar_dc='SIR_BN')
cfd.add_network('IDU', 'BA', m=5)
cfd.add_behaviour('cycle', be_type='Reincarnation', s_birth = 'Sus', s_death = 'Dead')
cfd.add_behaviour('transmission', be_type='NetShock', s_src = 'Inf', t_tar = 'Infect_Net', net='IDU')
cfd.add_behaviour('foi', be_type='ForeignShock', t_tar = 'Infect')

cfd.set_observations(states=['Inf'], behaviours=['foi'])

In [ ]:
lyo = da.new_layout('Lyo1')

lyo.add_entry('A', 'ABM_M2', {'Sus': 20, 'Inf': 10})
lyo.add_entry('B', 'ABM_M2', {'Sus': 20}, size=3)
lyo.add_entry('C', 'ABM_M5', {'Sus': 20, 'Inf': 10}, size=2)
lyo.add_relation('*@Inf', 'Lyo1@FOI')
lyo.add_relation('Lyo1@FOI', '*@foi')
lyo.set_observations(['B_2', 'B_1', '*'])

In [ ]:
mod, out = da.simulate('Lyo1', fr=0, to=10)
out.plot()
plt.show()

In [ ]:
pc, dc = generate_pc_dc(da.get_pc('pSIR'), da.get_dc('SIR_BN'))
mc = CoreODE(dc)
mc.Mods['FOI'] = InfectionFD('FOI', 'Infect', 'Inf')
mc.Mods['Dead'] = Reincarnation('Dead', s_death='Dead', s_birth='Sus')
model = ODEModel('SIR', mc, dt=0.01)
model.add_obs_transition('Infect')
model.add_obs_state('Alive')
model.add_obs_state('Inf')
model.add_obs_state('Sus')   
model.add_obs_behaviour('Dead')

In [ ]:
cfd = da.new_mc('EBM', 'CoreODE', tar_pc='pSIR', tar_dc='SIR_BN')
cfd.add_behaviour('FOI', be_type='InfectionDD', s_src = 'Inf', t_tar = 'Infect')
cfd.add_behaviour('Dead', be_type='Reincarnation', s_death='Dead', s_birth='Sus')
cfd.set_arguments('dt', 0.5)
cfd.set_observations(states=['Sus', 'Inf', 'Rec', 'Alive'], behaviours=['FOI'])


In [ ]:
mod, out = da.simulate('EBM', y0={'Sus': 95, 'Inf': 5}, fr=0 ,to=10, dt=1)
#da.copy_model(mod_src=mod)
out.plot()
plt.show()

In [ ]:
out = simulate(model, y0={'Sus': 95, 'Inf': 5}, fr=0 ,to=10, dt=0.1)
out.plot()
plt.show()

In [ ]:
mod.Obs.Obs